In [1]:
import os
import nrrd
import numpy as np
from scipy.ndimage import binary_dilation

In [2]:
import numpy as np
from skimage.color import gray2rgb, label2rgb
from skimage.segmentation import find_boundaries
from skimage.util import img_as_float
from skimage.morphology import dilation, square
import random

def mark_boundaries_color(image, label_img, color=None, outline_color=None, mode='outer', background_label=0, dilation_size=1):
    """Return image with boundaries between labeled regions highlighted with consistent colors derived from labels.

    Parameters:
    - image: Input image.
    - label_img: Image with labeled regions.
    - color: Ignored in this version.
    - outline_color: If specified, use this color for the outline. Otherwise, use the same as boundary.
    - mode: Choose 'inner', 'outer', or 'thick' to define boundary type.
    - background_label: Label to be treated as the background.
    - dilation_size: Size of the dilation square for the boundaries.

    Returns:
    - Image with boundaries highlighted.
    """
    # Ensure input image is in float and has three channels
    float_dtype = np.float32  # Use float32 for efficiency
    marked = img_as_float(image, force_copy=True).astype(float_dtype, copy=False)
    if marked.ndim == 2:
        marked = gray2rgb(marked)

    # Create a color map normalized by the number of unique labels
    unique_labels = np.unique(label_img)
    color_map = plt.get_cmap('nipy_spectral')  # You can change 'nipy_spectral' to any other colormap

    # Find boundaries and apply colors
    boundaries = find_boundaries(label_img, mode=mode, background=background_label)
    for label in unique_labels:
        if label == background_label:
            continue
        # Normalize label value to the range of the colormap
        normalized_color = color_map(label / np.max(unique_labels))[:3]  # Get RGB values only
        label_boundaries = find_boundaries(label_img == label, mode=mode)
        label_boundaries = dilation(label_boundaries, square(dilation_size))
        marked[label_boundaries] = normalized_color
        if outline_color is not None:
            outlines = dilation(label_boundaries, square(dilation_size + 1))
            marked[outlines] = outline_color
        else:
            marked[label_boundaries] = normalized_color

    return marked


def consistent_color(label):
    """Generate a consistent color for a given label using a hash function."""
    random.seed(hash(label))
    return [random.random() for _ in range(3)]

def mark_boundaries_multicolor(image, label_img, color=None, outline_color=None, mode='outer', background_label=0, dilation_size=1):
    """Return image with boundaries between labeled regions highlighted with consistent colors.

    Parameters are the same as in the original function but color is ignored if provided.
    """
    # Ensure input image is in float and has three channels
    float_dtype = np.float32  # Use float32 for efficiency
    marked = img_as_float(image, force_copy=True).astype(float_dtype, copy=False)
    if marked.ndim == 2:
        marked = gray2rgb(marked)

    # Generate consistent colors for each unique label in label_img
    unique_labels = np.unique(label_img)
    color_map = {label: consistent_color(label) for label in unique_labels if label != background_label}

    # Find boundaries and apply colors
    boundaries = find_boundaries(label_img, mode=mode, background=background_label)
    for label, color in color_map.items():
        label_boundaries = find_boundaries(label_img == label, mode=mode)
        label_boundaries = dilation(label_boundaries, square(dilation_size))
        if outline_color is not None:
            outlines = dilation(label_boundaries, square(dilation_size))
            marked[outlines] = outline_color
        marked[label_boundaries] = color

    return marked

def plot_segmentation_results(test_slice, segmentation):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Show marked boundary image
    axes[0].imshow(mark_boundaries(test_slice, np.array(segmentation)))
    axes[0].set_title("Marked Boundary")

    # Show unmarked boundary image
    axes[1].imshow(test_slice, cmap='gray')
    axes[1].set_title("Unmarked Boundary")

    plt.show()

In [3]:
import os
import nrrd
import numpy as np
from scipy.ndimage import binary_dilation, binary_closing

def process_nrrd_files(root_dir):
    for subdir, _, files in os.walk(root_dir):
        if os.path.basename(subdir) == 'original_labels':
            # Determine the corresponding 'label' directory
            label_dir = os.path.join(os.path.dirname(subdir), 'label')
            os.makedirs(label_dir, exist_ok=True)

            for file in files:
                if file.endswith('.nrrd'):
                    nrrd_path = os.path.join(subdir, file)
                    data, header = nrrd.read(nrrd_path)
                    
                    unique_values = np.unique(data)
                    unique_values = unique_values[unique_values != 0]  # Ignore background
                    
                    # Create an empty array for the result
                    result = np.zeros_like(data)
                    
                    for value in unique_values:
                        structure_mask = data == value
                        
                        # Fill small holes in the structure
                        closed_structure = structure_mask  # Keeping this as is
                        
                        # Dilate the closed structure
                        dilated_mask = binary_dilation(closed_structure, iterations=3)
                        
                        # Assign the border class and foreground class
                        border_class = dilated_mask & ~closed_structure
                        result[~border_class & dilated_mask] = 2  # Foreground class
                        result[border_class] = 1  # Border class
                    
                    # Save the processed array back to an .nrrd file
                    output_path = os.path.join(label_dir, f"tri_class_{file}")
                    nrrd.write(output_path, result, header)
                    print(f"Processed and saved: {output_path}")

In [4]:
root_directory = '/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/val'
process_nrrd_files(root_directory)

Processed and saved: /home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/val/label/tri_class_layers_1.nrrd


In [9]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
pred = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/val/label/tri_class_layers_1.nrrd')[0]
pred = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/val/raw/layers_1.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred[:,:,slice_index])
    else:
        plt.imshow(pred[slice_index,:,:], cmap='gray')
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [20]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import scipy.ndimage
pred = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/test/label/tri_class_layers_1.nrrd')[0]
foreground = pred == 2
fg, num_features = scipy.ndimage.label(foreground)
# pred = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/test/original_labels/layers_1.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(fg[:,slice_index,:])
    elif axis == 2:
        plt.imshow(fg[:,:,slice_index])
    else:
        plt.imshow(fg[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [51]:
import scipy.ndimage
def label_foreground_structures(input_array, min_size=100000): #use aggressize min_size to remove small structures, then decent overlap stride to get ones that are erronously cut off on the edges
    # Find connected components in the foreground (value 2)
    foreground = (input_array == 2)
    
    # Label connected components
    labeled_array, num_features = scipy.ndimage.label(foreground)
    
    # Measure the size of each connected component
    component_sizes = np.bincount(labeled_array.ravel())
    
    # Create a mask for components larger than the minimum size
    large_components = component_sizes >= min_size
    
    # Ensure background is not considered a component
    large_components[0] = False
    
    # Create a filtered array to hold only large components
    filtered_array = labeled_array.copy()
    
    # Set small components to 0 (background)
    filtered_array[~large_components[labeled_array]] = 0
    
    print(f"Number of connected foreground structures before filtering: {num_features}")
    print(f"Number of connected foreground structures after filtering: {np.max(filtered_array)}")
    
    return filtered_array


In [52]:
labeled_arr = label_foreground_structures(pred)
img_path = '/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/test/raw/layers_1.nrrd'
img, _ = nrrd.read(img_path)
gt_label = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/test/original_labels/layers_1.nrrd')[0]
print(np.unique(gt_label))

Number of connected foreground structures before filtering: 144
Number of connected foreground structures after filtering: 13
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14]


In [53]:
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(img[:,slice_index,:], labeled_arr[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(img[:,:,slice_index], labeled_arr[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(img[slice_index,:,:], labeled_arr[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [12]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
pred2 = nrrd.read('/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/test/original_labels/layers_1.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred2[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred2[:,:,slice_index])
    else:
        plt.imshow(pred2[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred2.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [10]:
import os
import h5py
import nrrd
import numpy as np

def create_hdf5_files(raw_dir, label_dir, output_dir, weight_dir=None):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for raw_filename in os.listdir(raw_dir):
        if raw_filename.endswith('.nrrd'):
            # Construct full file paths for raw and label
            raw_path = os.path.join(raw_dir, raw_filename)
            label_filename = f"tri_class_{raw_filename}"
            label_path = os.path.join(label_dir, label_filename)

            print(f"Processing: {raw_filename} and {label_filename}")
            
            if not os.path.exists(label_path):
                label_filename = "tri_class_"+'_'.join(raw_filename.split('_')[:-1])+'_label.nrrd'
                print(f"Label file not found for {raw_filename}, trying {label_filename}")
                label_path = os.path.join(label_dir, label_filename)
            if not os.path.exists(label_path):
                print(f"Label file not found for {raw_filename}, skipping.")
                continue
            
            # Read raw and label data
            raw_data, raw_header = nrrd.read(raw_path)
            label_data, label_header = nrrd.read(label_path)
            
            # Optionally read weight data
            weight_data = None
            if weight_dir:
                weight_path = os.path.join(weight_dir, label_filename)
                if os.path.exists(weight_path):
                    weight_data, weight_header = nrrd.read(weight_path)
            
            # Convert to numpy arrays
            raw_data = np.asarray(raw_data, dtype=np.float32)
            label_data = np.asarray(label_data, dtype=np.uint8)
            if weight_data is not None:
                weight_data = np.asarray(weight_data, dtype=np.float32)
            
            # Construct output file path
            output_filename = os.path.splitext(raw_filename)[0] + '.h5'
            output_path = os.path.join(output_dir, output_filename)
            
            # Create HDF5 file
            with h5py.File(output_path, 'w') as hdf5_file:
                hdf5_file.create_dataset('raw', data=raw_data, dtype='float32')
                hdf5_file.create_dataset('label', data=label_data, dtype='uint8')
                if weight_data is not None:
                    hdf5_file.create_dataset('weight', data=weight_data, dtype='float32')
            
            print(f"Processed and saved {raw_filename} and {label_filename} to {output_path}")

In [ ]:
sub_dir = 'test'
raw_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/raw'
label_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/label'
output_hdf5_path = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path)

In [11]:
sub_dir = 'val'
raw_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/raw'
label_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/label'
output_hdf5_path = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path)

Processing: layers_1.nrrd and tri_class_layers_1.nrrd
Processed and saved layers_1.nrrd and tri_class_layers_1.nrrd to /home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/val/dataset/layers_1.h5


In [64]:
sub_dir = 'train'
raw_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/raw'
label_directory = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/label'
output_hdf5_path = f'/home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path)

Processing: 3350_4000_8450_xyz_256_res1_s4_raw.nrrd and tri_class_3350_4000_8450_xyz_256_res1_s4_raw.nrrd
Label file not found for 3350_4000_8450_xyz_256_res1_s4_raw.nrrd, trying tri_class_3350_4000_8450_xyz_256_res1_s4_label.nrrd
Processed and saved 3350_4000_8450_xyz_256_res1_s4_raw.nrrd and tri_class_3350_4000_8450_xyz_256_res1_s4_label.nrrd to /home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/train/dataset/3350_4000_8450_xyz_256_res1_s4_raw.h5
Processing: layers_1.nrrd and tri_class_layers_1.nrrd
Processed and saved layers_1.nrrd and tri_class_layers_1.nrrd to /home/james/Documents/VS/pytorch-3dunet-instanceSeg/data/Vesuvius/train/dataset/layers_1.h5
